In [1]:
! pip install 'tritonclient[http]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.6/228.6 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 8.8 MB/s eta 0:00:008.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.7/786.7 kB 8.6 MB/s eta 0:00:009.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 7.2 MB/s eta 0:00:00
  Using cached greenlet-2.0.2-cp310-cp310-macosx_13_0_arm64.whl
  Created wheel for python-rapidjson: filename=python_rapidjson-1.10-cp310-cp310-macosx_13_0_arm64.whl size=197754 sha256=9db5178748e32fc5fe608f7012c1ee0abc61d838d8b56f78108960ea731ce4ef
  Stored 

In [2]:
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype

In [8]:
client = httpclient.InferenceServerClient(url="localhost:8000")

In [3]:
query = httpclient.InferInput("TEXT", shape=(1,) , datatype="BYTES",)

In [4]:
text_to_translate = "I am learning how to use Triton Server for Machine Learning"

In [5]:
query.set_data_from_numpy(np.asarray([text_to_translate], dtype=object))

In [6]:
outputs = httpclient.InferRequestedOutput("last_hidden_state", binary_data=False)

In [9]:
results = client.infer(model_name="ensemble_model", inputs=[query], outputs=[outputs])
inference_output = results.as_numpy('last_hidden_state')

In [10]:
inference_output

array([[[ 7.7679253e-01,  1.4532851e+00, -1.8967432e-01, ...,
          5.7005125e-01,  8.7255359e-01,  1.6838910e-01],
        [-9.8925894e-01,  1.6409941e-03,  2.0253782e-01, ...,
          8.2247579e-01, -5.1777565e-01, -5.4934025e-01],
        [-5.9773749e-01, -2.0363115e-01, -1.1606464e+00, ...,
          1.3142958e+00, -4.0673012e-01, -7.6064211e-01],
        ...,
        [ 7.5196075e-01, -1.8917035e-01, -6.6133803e-01, ...,
         -8.2274443e-01,  4.9503481e-01, -1.0956843e+00],
        [-4.6586528e-01, -1.7450033e-01,  2.3366473e+00, ...,
          6.8499488e-01, -1.9554515e+00, -6.5189815e-01],
        [ 4.4324290e-02,  3.5424232e-02,  3.5430957e-02, ...,
          2.1137428e-02,  4.5173872e-02, -3.3297587e-02]]], dtype=float32)

With this we have the transformer working with the encoder part of the model, we need now to learn how to use the decoder and the decoder part with attention

 docker run --rm -p 8000:8000 -p 8001:8001 -p 8002:8002 -v ${PWD}/triton_model_repository:/models nvcr.io/nvidia/tritonserver:23.06-py3 bash -c "pip install transformers==4.30.2  sentencepiece==0.1.99 && tritonserver --model-repository=/models"